In [33]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone as PC
from pinecone import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [23]:
PINECONE_API_KEY="8273f958-c1a5-4497-8f8c-99fff6730a73"
PINECONE_API_ENV="us-east-1-aws"

In [3]:
def load_pdf(data):
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    document=loader.load()
    return document

In [4]:
extracted_data=load_pdf("data/")

Create text chunks

In [5]:
def text_split(extracted_data):
    text_spliter=RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
    text_chunks=text_spliter.split_documents(extracted_data)
    return text_chunks

In [6]:
text_chunks=text_split(extracted_data)
print("Length of my chunks: ",len(text_chunks))

Length of my chunks:  5460


In [7]:
# Download embedding model
def download_huggingface_embedding():
    embedding = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

In [8]:
embedding=download_huggingface_embedding()

d:\Machine_learning\LLMs\Maths-chatbot-using-Llama2\mchatbot\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
from pinecone.grpc import PineconeGRPC
pc = PineconeGRPC(api_key=PINECONE_API_KEY)

pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 384,
              'host': 'maths-chatbot-8m5l21j.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'maths-chatbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [35]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [36]:
#Initializing the Pinecone
# pinecone.init(api_key=PINECONE_API_KEY,
#               environment=PINECONE_API_ENV)
pc=Pinecone(api_key=PINECONE_API_KEY)
index_name="maths-chatbot"
docsearch=PC.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

In [37]:
docsearch